In [1]:
import os
import numpy as np
from PIL import Image
import glob
import random
from keras.utils import Sequence

In [2]:
random.seed(1143)

### Function to prepare training and validation data

In [4]:
def preparing_training_data_1(hazefree_images_dir, hazeeffected_images_dir):


    train_data = []
    validation_data = []
    
    hazy_data = glob.glob(hazeeffected_images_dir + "*.jpg")

    data_holder = {}

    for h_image in hazy_data:
        h_image = h_image.split("\\")[-1]
        id_ = h_image.split("_")[0] + "_" + h_image.split("_")[1] + ".jpg"
        if id_ in data_holder.keys():
            data_holder[id_].append(h_image)
        else:
            data_holder[id_] = []
            data_holder[id_].append(h_image)


    train_ids = []
    val_ids = []

    num_of_ids = len(data_holder.keys())
    print(num_of_ids)
    for i in range(num_of_ids):
        if i < num_of_ids*9/10:
            train_ids.append(list(data_holder.keys())[i])
        else:
            val_ids.append(list(data_holder.keys())[i])


    for id_ in list(data_holder.keys()):

        if id_ in train_ids:
            
            for hazy_image in data_holder[id_]:
                train_data.append([hazefree_images_dir + id_, hazeeffected_images_dir + hazy_image])


        else:
            for hazy_image in data_holder[id_]:
                validation_data.append([hazefree_images_dir + id_, hazeeffected_images_dir + hazy_image])



    random.shuffle(train_data)
    random.shuffle(validation_data)

    return train_data, validation_data

In [6]:

preparing_training_data_1('E:\Keras\Dataset\Hazy_data\Hazy_free\\', 'E:\Keras\Dataset\Hazy_data\Hazy_affected\\')

1449


([['E:\\Keras\\Dataset\\Hazy_data\\Hazy_free\\NYU2_1130.jpg',
   'E:\\Keras\\Dataset\\Hazy_data\\Hazy_affected\\NYU2_1130_2_2.jpg'],
  ['E:\\Keras\\Dataset\\Hazy_data\\Hazy_free\\NYU2_1221.jpg',
   'E:\\Keras\\Dataset\\Hazy_data\\Hazy_affected\\NYU2_1221_7_3.jpg'],
  ['E:\\Keras\\Dataset\\Hazy_data\\Hazy_free\\NYU2_625.jpg',
   'E:\\Keras\\Dataset\\Hazy_data\\Hazy_affected\\NYU2_625_7_2.jpg'],
  ['E:\\Keras\\Dataset\\Hazy_data\\Hazy_free\\NYU2_735.jpg',
   'E:\\Keras\\Dataset\\Hazy_data\\Hazy_affected\\NYU2_735_4_2.jpg'],
  ['E:\\Keras\\Dataset\\Hazy_data\\Hazy_free\\NYU2_1417.jpg',
   'E:\\Keras\\Dataset\\Hazy_data\\Hazy_affected\\NYU2_1417_4_3.jpg'],
  ['E:\\Keras\\Dataset\\Hazy_data\\Hazy_free\\NYU2_1309.jpg',
   'E:\\Keras\\Dataset\\Hazy_data\\Hazy_affected\\NYU2_1309_1_1.jpg'],
  ['E:\\Keras\\Dataset\\Hazy_data\\Hazy_free\\NYU2_799.jpg',
   'E:\\Keras\\Dataset\\Hazy_data\\Hazy_affected\\NYU2_799_3_1.jpg'],
  ['E:\\Keras\\Dataset\\Hazy_data\\Hazy_free\\NYU2_262.jpg',
   'E:\\Keras\

In [28]:
class HazyDataGenerator1(Sequence):
    
    def __init__(self, hazefree_images_dir, hazeeffected_images_dir, mode='train'):
        self.train_data, self.validation_data = preparing_training_data_1(hazefree_images_dir, hazeeffected_images_dir)
        self.mode = mode
        if self.mode == 'train':
            self.data = self.train_data
            print("Number of Training Images:", len(self.train_data))
        else:
            self.data = self.validation_data
            print("Number of Validation Images:", len(self.validation_data))

            
            
        # Use the ImageDataGenerator for data augmentation (optional but recommended for better generalization)
      # self.data_generator = ImageDataGenerator(
           #rescale=1.0 / 255.0  # Normalize the pixel values to the range [0, 1]
            # Add more data augmentation options if needed (e.g., rotation, width/height shift, etc.)
      # )

    
    
    def __len__(self):
        return int(np.ceil(len(self.data) / self.batch_size))

    
    
    def __getitem__(self, idx):
        batch_data = self.data[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_hazefree_images = []
        batch_hazy_images = []

        for hazefree_image_path, hazy_image_path in batch_data:
            hazefree_image = Image.open(hazefree_image_path)
            hazy_image = Image.open(hazy_image_path)

            # Resize the images to the desired size (480, 640)
            hazefree_image = hazefree_image.resize((480, 640), Image.ANTIALIAS)
            hazy_image = hazy_image.resize((480, 640), Image.ANTIALIAS)

            # Convert the images to numpy arrays
            hazefree_image = np.array(hazefree_image)/255.0
            hazy_image = np.array(hazy_image)/255.0

            # Perform data augmentation (optional but recommended)
            hazefree_image = self.data_generator.random_transform(hazefree_image)
            hazy_image = self.data_generator.random_transform(hazy_image)

            batch_hazefree_images.append(hazefree_image)
            batch_hazy_images.append(hazy_image)

        return np.array(batch_hazefree_images), np.array(batch_hazy_images)

    
    
    def on_epoch_end(self):
        random.shuffle(self.data)

In [52]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import glob
import random
from tensorflow.keras.utils import Sequence
from PIL import Image

class HazyDataGenerator(Sequence):
    
    def __init__(self, hazefree_images_dir, hazeeffected_images_dir, mode='train', batch_size=32):
        self.train_data, self.validation_data = preparing_training_data_1(hazefree_images_dir, hazeeffected_images_dir)
        self.mode = mode
        self.batch_size = batch_size
        if self.mode == 'train':
            self.data = self.train_data
            print("Number of Training Images:", len(self.train_data))
        else:
            self.data = self.validation_data
            print("Number of Validation Images:", len(self.validation_data))

        # Use the ImageDataGenerator for data augmentation (optional but recommended for better generalization)
        self.data_generator = ImageDataGenerator(
            rescale=1.0 / 255.0,  # Normalize the pixel values to the range [0, 1]
            # Add more data augmentation options if needed (e.g., rotation, width/height shift, etc.)
        )

    def __len__(self):
        return int(np.ceil(len(self.data) / self.batch_size))

    def __getitem__(self, idx):
        batch_data = self.data[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_hazefree_images = []
        batch_hazy_images = []

        for hazefree_image_path, hazy_image_path in batch_data:
            hazefree_image = Image.open(hazefree_image_path)
            hazy_image = Image.open(hazy_image_path)

            # Resize the images to the desired size (480, 640)
            hazefree_image = hazefree_image.resize((480, 640), Image.ANTIALIAS)
            hazy_image = hazy_image.resize((480, 640), Image.ANTIALIAS)

            # Convert the images to numpy arrays and float32 data type
            hazefree_image = np.array(hazefree_image) / 255.0
            hazy_image = np.array(hazy_image) / 255.0

            # Perform data augmentation (optional but recommended)
            hazefree_image = self.data_generator.random_transform(hazefree_image.astype(np.float32))
            hazy_image = self.data_generator.random_transform(hazy_image.astype(np.float32))

            batch_hazefree_images.append(hazefree_image)
            batch_hazy_images.append(hazy_image)

        return np.array(batch_hazefree_images), np.array(batch_hazy_images)

### Model Building

In [53]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Concatenate, ReLU
from tensorflow.keras.models import Model

In [54]:
class LightDehazeNet(Model):

    def __init__(self):
        super(LightDehazeNet, self).__init__()

        # LightDehazeNet Architecture 
        self.relu = ReLU()

        self.e_conv_layer1 = Conv2D(8, kernel_size=(1, 1), padding='same')
        self.e_conv_layer2 = Conv2D(8, kernel_size=(3, 3), padding='same')
        self.e_conv_layer3 = Conv2D(8, kernel_size=(5, 5), padding='same')
        self.e_conv_layer4 = Conv2D(16, kernel_size=(7, 7), padding='same')
        self.e_conv_layer5 = Conv2D(16, kernel_size=(3, 3), padding='same')
        self.e_conv_layer6 = Conv2D(16, kernel_size=(3, 3), padding='same')
        self.e_conv_layer7 = Conv2D(32, kernel_size=(3, 3), padding='same')
        self.e_conv_layer8 = Conv2D(3, kernel_size=(3, 3), padding='same')
        
    def call(self, img):
        pipeline = []
        pipeline.append(img)

        conv_layer1 = self.relu(self.e_conv_layer1(img))
        conv_layer2 = self.relu(self.e_conv_layer2(conv_layer1))
        conv_layer3 = self.relu(self.e_conv_layer3(conv_layer2))

        # concatenating conv1 and conv3
        concat_layer1 = Concatenate()([conv_layer1, conv_layer3])
        
        conv_layer4 = self.relu(self.e_conv_layer4(concat_layer1))
        conv_layer5 = self.relu(self.e_conv_layer5(conv_layer4))
        conv_layer6 = self.relu(self.e_conv_layer6(conv_layer5))

        # concatenating conv4 and conv6
        concat_layer2 = Concatenate()([conv_layer4, conv_layer6])
        
        conv_layer7 = self.relu(self.e_conv_layer7(concat_layer2))

        # concatenating conv2, conv5, and conv7
        concat_layer3 = Concatenate()([conv_layer2, conv_layer5, conv_layer7])
        
        conv_layer8 = self.relu(self.e_conv_layer8(concat_layer3))

        dehaze_image = self.relu((conv_layer8 * img) - conv_layer8 + 1) 
        # J(x) = clean_image, k(x) = x8, I(x) = x, b = 1

        return dehaze_image

In [55]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

In [56]:
# Set the paths to the hazy and haze-free image directories
train_hazy_images_dir = "E:\Keras\Dataset\Hazy_data\Hazy_affected\\"
train_original_images_dir = "E:\Keras\Dataset\Hazy_data\Hazy_free\\"

In [57]:

# Set hyperparameters
learning_rate = 0.001
batch_size = 2
epochs = 10

In [58]:
# Create the model
model = LightDehazeNet()

In [59]:
# Create custom data generators
train_data_generator = HazyDataGenerator(
    hazefree_images_dir=train_original_images_dir,
    hazeeffected_images_dir=train_hazy_images_dir,
    mode='train'
)

1449
Number of Training Images: 24443


In [60]:
# Create custom data generators
val_data  = HazyDataGenerator(
    hazefree_images_dir=train_original_images_dir,
    hazeeffected_images_dir=train_hazy_images_dir, mode = 'val'
)

1449
Number of Validation Images: 2813


In [61]:
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.optimizers import Adam

learning_rate = 0.001
weight_decay = 0.0001

# Define the mean squared error (MSE) loss function
criterion = mean_squared_error

# Create the Adam optimizer
optimizer = Adam(learning_rate=learning_rate)

# Compile the model with the chosen optimizer and loss function
model.compile(optimizer=optimizer, loss=criterion)

In [ ]:
# Train the model
history = model.fit(
    train_data_generator,
    epochs=30,
    steps_per_epoch=round(len(train_data_generator)/20),
    validation_data=val_data,
    validation_steps=round(len(val_data)/20),
    batch_size=20
)

C:\Users\tanvi\AppData\Local\Temp\ipykernel_12040\2170823626.py:40: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  hazefree_image = hazefree_image.resize((480, 640), Image.ANTIALIAS)
C:\Users\tanvi\AppData\Local\Temp\ipykernel_12040\2170823626.py:41: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  hazy_image = hazy_image.resize((480, 640), Image.ANTIALIAS)


Epoch 1/30
38/38 [==============================] - 972s 25s/step - loss: 0.0424 - val_loss: 0.0178
Epoch 2/30
38/38 [==============================] - 982s 26s/step - loss: 0.0146 - val_loss: 0.0147
Epoch 3/30
38/38 [==============================] - 1048s 28s/step - loss: 0.0135 - val_loss: 0.0145
Epoch 4/30
38/38 [==============================] - 1116s 29s/step - loss: 0.0123 - val_loss: 0.0135
Epoch 5/30
38/38 [==============================] - 1070s 28s/step - loss: 0.0126 - val_loss: 0.0136
Epoch 6/30
38/38 [==============================] - 1053s 28s/step - loss: 0.0123 - val_loss: 0.0139
Epoch 7/30
38/38 [==============================] - 1015s 27s/step - loss: 0.0128 - val_loss: 0.0154
Epoch 8/30
38/38 [==============================] - 1021s 27s/step - loss: 0.0137 - val_loss: 0.0136
Epoch 9/30
38/38 [==============================] - 1012s 27s/step - loss: 0.0123 - val_loss: 0.0136
Epoch 10/30
38/38 [==============================] - 1011s 27s/step - loss: 0.0119 - val_loss

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from PIL import Image
import tensorflow as tf

def image_haze_removal(input_image, ld_net):

    # Preprocess the input image
    hazy_image = img_to_array(input_image) / 255.0
    hazy_image = np.expand_dims(hazy_image, axis=0)

    # Perform haze removal using the Keras model
    dehaze_image = ld_net.predict(hazy_image)

    # Postprocess the output image
    dehaze_image = np.squeeze(dehaze_image, axis=0)
    dehaze_image = np.clip(dehaze_image, 0, 1)  # Clip values to [0, 1] range

    # Convert the output back to PIL Image format
    dehaze_image = array_to_img(dehaze_image * 255.0)

    return dehaze_image
